In [ ]:
! pip install accelerate peft bitsandbytes transformers trl auto-gptq optimum

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.3/168.3 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.1/139.1 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 26.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 403.3/403.3 kB 19.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 16.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.9/78.9 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 30.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.2/12.2 MB 27.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
from huggingface_hub import notebook_login
notebook_login()


In [ ]:

import torch
from datasets import load_dataset, Dataset
from peft import LoraConfig, AutoPeftModelForCausalLM, get_peft_model, prepare_model_for_kbit_training
from transformers import AutoModelForCausalLM, AutoTokenizer, GPTQConfig, TrainingArguments
from trl import SFTTrainer
import os

In [ ]:


def finetune_xwin_lm():
    data = load_dataset("Aneeth/job_descp_5k_samples", split="train")
    data_df = data.to_pandas()
    data_df = data_df[:10000]
    data_df["text"] = data_df[["user_prompt", "model_response"]].apply(lambda x: "###Human: " + "Generate a job description from the following input." + "\n" + x["user_prompt"] + "\n###Assistant: " +x["model_response"], axis=1)
    print(data_df.iloc[0])
    data = Dataset.from_pandas(data_df)
    tokenizer = AutoTokenizer.from_pretrained("TheBloke/Xwin-LM-7B-V0.1-GPTQ")
    tokenizer.pad_token = tokenizer.eos_token
    quantization_config_loading = GPTQConfig(bits=4, disable_exllama=True)
    model = AutoModelForCausalLM.from_pretrained(
                              "TheBloke/Xwin-LM-7B-V0.1-GPTQ",
                              quantization_config=quantization_config_loading,
                              device_map="auto"
                          )
    model.config.use_cache=False
    model.config.pretraining_tp=1
    model.gradient_checkpointing_enable()
    model = prepare_model_for_kbit_training(model)
    peft_config = LoraConfig(
        r=4, lora_alpha=1, lora_dropout=0.1, bias="none", task_type="CAUSAL_LM"
    )
    model = get_peft_model(model, peft_config)
    training_arguments = TrainingArguments(
        output_dir="xwin-job_descp_5ksamples",
        per_device_train_batch_size=8,
        gradient_accumulation_steps=1,
        optim="paged_adamw_32bit",
        learning_rate=1e-4,
        lr_scheduler_type="cosine",
        save_strategy="steps",
        logging_steps=50,
        num_train_epochs=1,
        # max_steps=250,
        push_to_hub=True
    )
    trainer = SFTTrainer(
        model=model,
        train_dataset=data,
        peft_config=peft_config,
        dataset_text_field="text",
        args=training_arguments,
        tokenizer=tokenizer,
        packing=False,
        max_seq_length=5000
    )
    trainer.train()
    trainer.push_to_hub()

if __name__ == "__main__":
    finetune_xwin_lm()


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/5000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/100 [00:00<?, ? examples/s]

Unnamed: 0                                                        0
user_prompt       Generate a job description for a Systems Admin...
model_response    Job Description: Systems Administrator (Insura...
text              ###Human: Generate a job description from the ...
Name: 0, dtype: object


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


...

You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss
50,1.426500
100,0.979000


Step,Training Loss
50,1.426500
100,0.979000
150,0.682000
200,0.538200
250,0.462100
300,0.409600
350,0.377800
400,0.357000
450,0.333300
500,0.321700


/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


KeyboardInterrupt: ignored

In [ ]:
!git lfs install
!git clone https://huggingface.co/Aneeth/xwin-job_descp_5ksamples

Git LFS initialized.
Cloning into 'xwin-job_descp_5ksamples'...
remote: Enumerating objects: 17, done.
remote: Counting objects: 100% (14/14), done.
remote: Compressing objects: 100% (14/14), done.
remote: Total 17 (delta 0), reused 0 (delta 0), pack-reused 3
Unpacking objects: 100% (17/17), 481.03 KiB | 2.42 MiB/s, done.


In [ ]:

from peft import AutoPeftModelForCausalLM
from transformers import GenerationConfig
from transformers import AutoTokenizer
import torch
tokenizer = AutoTokenizer.from_pretrained("/content/xwin-job_descp_5ksamples")

inputs = tokenizer("""
###Instruction: Generate a job description from the following input.
Generate a job description for data scientist role. the skills needed are python, AI/ML, GenAI and qualification should be atleast bachelors in computer science or data science.the candidate should be interested in developing machine learning models and interested working in cloud environments.
###Response: """, return_tensors="pt").to("cuda")

model = AutoPeftModelForCausalLM.from_pretrained(
    "/content/xwin-job_descp_5ksamples",
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map="cuda")

generation_config = GenerationConfig(
    do_sample = True,
    top_k=4,
    temperature=0.3,
    max_new_tokens=5000
)
outputs = model.generate(**inputs, generation_config=generation_config)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))


###Instruction: Generate a job description from the following input.
Generate a job description for data scientist role. the skills needed are python, AI/ML, GenAI and qualification should be atleast bachelors in computer science or data science.the candidate should be interested in developing machine learning models and interested working in cloud environments.
###Response: 
Job Description: Data Scientist

We are currently seeking a skilled Data Scientist to join our team. As a Data Scientist, you will be responsible for developing machine learning models and algorithms, using Python and other programming languages. You will also be working with large datasets and performing data analysis.

Responsibilities:
- Develop machine learning models and algorithms using Python and other programming languages.
- Work with large datasets and perform data analysis.

Qualifications:
- Bachelor's degree in Computer Science or Data Science.
- Demonstrated expertise in Python and AI/ML.
- Knowledg

In [ ]:
import zipfile
import os

# Define the folder you want to zip
folder_to_zip = '/content/xwin-job_descp_5ksamples/checkpoint-500'  # Replace 'sample_folder' with your folder name

# Define the name for the zip file
zip_file_name = 'xwin_model.zip'  # Replace 'sample_folder.zip' with your desired zip file name

# Create a ZipFile object
with zipfile.ZipFile(zip_file_name, 'w', zipfile.ZIP_DEFLATED) as zipf:
    # Walk through each file in the directory
    for root, dirs, files in os.walk(folder_to_zip):
        for file in files:
            # Write the file into the zip file
            zipf.write(os.path.join(root, file), os.path.relpath(os.path.join(root, file), folder_to_zip))

print(f"Folder '{folder_to_zip}' zipped successfully to '{zip_file_name}'")


Folder '/content/xwin-job_descp_5ksamples/checkpoint-500' zipped successfully to 'xwin_model.zip'
